## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    import seaborn as sns
    sns.set_style('whitegrid')
except ImportError:
    sns = None
import logging

# Ensure repo root is on sys.path (works from repo root or notebooks/)
repo_root = Path.cwd()
if not (repo_root / 'src').exists():
    repo_root = repo_root.parent
sys.path.insert(0, str(repo_root))

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split

from src.data_acquisition import KaggleDataAcquisition
from src.datasets import NovozymesDataset
from src.models.multimodal import VibroStructuralModel
from src.training import Trainer, MetricComputer
from src.utils import Logger, set_seed, get_device, batch_collate_function

logger = Logger.setup('QDD-Novozymes', level=logging.INFO)
set_seed(42)
device = get_device()

plt.rcParams['figure.figsize'] = (14, 6)

data_kaggle_dir = repo_root / 'data' / 'kaggle'
spectral_dir = repo_root / 'data' / 'spectral'
checkpoints_dir = repo_root / 'checkpoints'
data_kaggle_dir.mkdir(parents=True, exist_ok=True)
spectral_dir.mkdir(parents=True, exist_ok=True)
checkpoints_dir.mkdir(parents=True, exist_ok=True)

logger.info('Setup complete!')


## 2. Download and Explore Novozymes Data

In [ ]:
# Download competition data (requires kaggle CLI credentials)
logger.info("Downloading Novozymes competition data...")
kaggle_acq = KaggleDataAcquisition(output_dir=str(data_kaggle_dir))

# Note: Uncomment to download
# train_csv, test_csv, struct_pdb = kaggle_acq.download_novozymes()

# For this demo, check if files exist
train_csv = data_kaggle_dir / 'train.csv'
test_csv = data_kaggle_dir / 'test.csv'
struct_pdb = data_kaggle_dir / 'wildtype_structure_prediction_af2.pdb'

if not train_csv.exists():
    logger.warning("Train data not found. Please run: kaggle competitions download -c novozymes-enzyme-stability-prediction -p ./data/kaggle")
else:
    logger.info("Novozymes data found!")

## 3. Load and Explore Data

In [ ]:
# Load training data (or fall back to a small demo dataset)
if train_csv.exists():
    df_train = pd.read_csv(train_csv)
else:
    logger.warning("train.csv not found; using a demo dataset so the notebook can run end-to-end.")
    df_train = pd.DataFrame({
        'seq_id': [f'demo_{i}' for i in range(200)],
        'protein_sequence': ['ACDEFGHIKLMNPQRSTVWY' for _ in range(200)],
        'pH': np.random.choice([6.0, 7.0, 8.0], size=200),
        'tm': np.random.normal(loc=50.0, scale=5.0, size=200),
    })
    # Persist demo files so downstream cells can run unchanged
    df_train.to_csv(train_csv, index=False)
    if not test_csv.exists():
        df_demo_test = df_train.drop(columns=['tm']).head(50)
        df_demo_test.to_csv(test_csv, index=False)
logger.info(f"Training data shape: {df_train.shape}")
logger.info(f"\nFirst few rows:")
print(df_train.head())

logger.info(f"\nData info:")
print(df_train.info())

logger.info(f"\nTm statistics:")
print(df_train['tm'].describe())

In [ ]:
# Visualize Tm distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df_train['tm'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Melting Temperature (°C)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Tm Values', fontsize=13, fontweight='bold')
axes[0].grid(alpha=0.3)

# By pH
for ph in df_train['pH'].unique():
    mask = df_train['pH'] == ph
    axes[1].scatter(df_train[mask]['pH'], df_train[mask]['tm'], 
                   alpha=0.5, s=30, label=f'pH={ph}')
axes[1].set_xlabel('pH', fontsize=12)
axes[1].set_ylabel('Tm (°C)', fontsize=12)
axes[1].set_title('Tm vs pH', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

logger.info("Data exploration complete!")

## 4. Precompute Spectral Data

In [ ]:
# For efficiency, precompute spectra for all mutations
# In full implementation, would use mass-perturbation NMA

logger.info("Precomputing spectral data...")

if struct_pdb.exists():
    try:
        # Load wildtype structure
        import prody as pr
        wt_structure = pr.parsePDB(str(struct_pdb))
        logger.info(f"Wildtype structure: {wt_structure.numResidues()} residues")

        # Compute WT spectrum (cached)
        from src.nma_analysis import ANMAnalyzer
        from src.spectral_generation import SpectralGenerator

        anm = ANMAnalyzer(wt_structure, cutoff=15.0)
        freqs, _ = anm.compute_modes(k=50)
        s_vib = anm.compute_vibrational_entropy(k=50)

        sg = SpectralGenerator(freq_min=0, freq_max=500, n_points=1000)
        wt_spectrum = sg.generate_dos(freqs, broadening=5.0)

        # Save for dataset loading
        np.save(spectral_dir / 'wt_spectrum.npy', wt_spectrum)
        logger.info(f"WT spectrum saved. S_vib = {s_vib:.2f} J/(mol·K)")
    except Exception as e:
        logger.error(f"WT spectrum precompute failed: {e}")
        logger.info("Using synthetic spectrum")
        np.save(spectral_dir / 'wt_spectrum.npy', np.random.randn(1000))
else:
    logger.warning("Wildtype structure not found. Using synthetic spectra.")
    np.save(spectral_dir / 'wt_spectrum.npy', np.random.randn(1000))

## 5. Create Dataset and DataLoaders

In [ ]:
# Create Novozymes dataset
logger.info("Creating Novozymes dataset...")

try:
    dataset = NovozymesDataset(
        csv_file=str(train_csv),
        structure_file=str(struct_pdb),
        spectra_dir=str(spectral_dir),
        include_updates=True
    )
    logger.info(f"Dataset created: {len(dataset)} samples")
except:
    logger.warning("Novozymes dataset creation failed. Using dummy dataset for demo.")
    # Create dummy dataset for demonstration
    dataset = None

if dataset is not None:
    # Split into train/val/test (robust for small demo datasets)
    n_total = len(dataset)
    train_size = max(1, int(0.7 * n_total))
    val_size = max(1, int(0.15 * n_total))
    test_size = n_total - train_size - val_size
    if test_size < 1:
        test_size = 1
        train_size = max(1, n_total - val_size - test_size)
    
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size]
    )
    
    logger.info(f"Split: train={train_size}, val={val_size}, test={test_size}")
    
    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=32,
        shuffle=True,
        num_workers=0,
        collate_fn=batch_collate_function,
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=0,
        collate_fn=batch_collate_function,
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=0,
        collate_fn=batch_collate_function,
    )
    
    logger.info("DataLoaders created successfully!")
else:
    logger.info("Demo mode: skipping DataLoader creation")

## 6. Initialize Model and Training

In [ ]:
# Initialize model
logger.info("Initializing Vibro-Structural model...")

model = VibroStructuralModel(
    latent_dim=128,
    gnn_input_dim=24,
    fusion_type='bilinear',
    dropout=0.2,
    num_go_terms=10000  # Not used for Novozymes
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
logger.info(f"Model: {total_params:,} total parameters, {trainable_params:,} trainable")

# Setup training
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
try:
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
except TypeError:
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
loss_fn = nn.MSELoss()  # L2 loss for regression

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    checkpoint_dir=str(checkpoints_dir)
)

logger.info("Training setup complete!")

## 7. Train Model

In [ ]:
# Train model (requires actual dataset)
if dataset is not None:
    logger.info("Starting training...")
    
    best_loss = trainer.fit(
        train_loader=train_loader,
        val_loader=val_loader,
        loss_fn=loss_fn,
        epochs=10,
        metric_fn=MetricComputer.spearman_correlation,
        early_stopping_patience=5,
        task='novozymes'
    )
    
    logger.info(f"\nTraining complete! Best validation loss: {best_loss:.4f}")
else:
    logger.info("Demo mode: Training skipped. Use actual Novozymes data to train.")

## 8. Evaluate and Predict

In [ ]:
if dataset is not None:
    logger.info("Evaluating on test set...")
    
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            graph = batch['graph'].to(device)
            spectra = batch['spectra'].to(device)
            labels = batch['labels'].to(device)
            
            global_features = None
            if 'global_features' in batch:
                global_features = batch['global_features'].to(device)
            
            outputs = model(graph, spectra, global_features, task='novozymes')
            
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    all_preds = np.concatenate(all_preds, axis=0).squeeze()
    all_labels = np.concatenate(all_labels, axis=0)
    
    # Compute metrics
    spearman = MetricComputer.spearman_correlation(all_preds, all_labels)
    mae = MetricComputer.mean_absolute_error(all_preds, all_labels)
    mse = MetricComputer.mean_squared_error(all_preds, all_labels)
    
    logger.info(f"\nTest Set Metrics:")
    logger.info(f"  Spearman Correlation: {spearman:.4f}")
    logger.info(f"  MAE: {mae:.4f}")
    logger.info(f"  MSE: {mse:.4f}")

    # Generate Kaggle-style submission for test.csv (if present)
    if test_csv.exists():
        logger.info("Generating submission predictions for test.csv...")
        df_test = pd.read_csv(test_csv)
        test_dataset = NovozymesDataset(
            csv_file=str(test_csv),
            structure_file=str(struct_pdb),
            spectra_dir=str(spectral_dir),
            include_updates=False,
        )
        test_loader_submit = DataLoader(
            test_dataset,
            batch_size=32,
            shuffle=False,
            num_workers=0,
            collate_fn=batch_collate_function,
        )

        preds = []
        with torch.no_grad():
            for batch in test_loader_submit:
                graph = batch['graph'].to(device)
                spectra = batch['spectra'].to(device)
                global_features = batch.get('global_features')
                if global_features is not None:
                    global_features = global_features.to(device)
                out = model(graph, spectra, global_features, task='novozymes').squeeze(-1)
                preds.append(out.cpu().numpy())

        preds = np.concatenate(preds, axis=0)
        submission = pd.DataFrame({'seq_id': df_test['seq_id'].values, 'tm': preds})
        out_path = data_kaggle_dir / 'submission.csv'
        submission.to_csv(out_path, index=False)
        logger.info(f"Saved submission: {out_path}")
else:
    logger.info("Demo mode: Evaluation skipped.")

## 9. Summary and Next Steps

In [ ]:
logger.info("\n" + "="*60)
logger.info("Novozymes Competition Execution Summary")
logger.info("="*60)
logger.info(f"Competition: Novozymes Enzyme Stability Prediction")
logger.info(f"Task: Predict melting temperature (Tm) from structure")
logger.info(f"Metric: Spearman rank correlation")
logger.info(f"Approach: Vibro-structural multimodal model")
logger.info(f"\nModel: VibroStructuralModel")
logger.info(f"  - GNN branch: Structural graph encoding")
logger.info(f"  - CNN branch: Spectral fingerprint encoding")
logger.info(f"  - Fusion: Bilinear transformation")
logger.info(f"  - Head: Regression MLP for Tm prediction")
logger.info(f"\nNext steps:")
logger.info(f"  1. Download full Novozymes dataset")
logger.info(f"  2. Precompute all mutation spectra (mass-perturbation NMA)")
logger.info(f"  3. Implement mutation-specific delta features")
logger.info(f"  4. Train on full dataset (4,000+ mutations)")
logger.info(f"  5. Evaluate with Spearman correlation")
logger.info(f"  6. Generate test set predictions")
logger.info(f"  7. Submit to Kaggle competition")
logger.info("="*60)